#Restic Ransomware Simulation & Auto‑Recovery Lab

Integrates automated Restic snapshots, simulates a ransomware attack, and then verifies and restores the latest clean baseline—all without manual intervention.

##Summary

This notebook spins up a self‑contained playground that

##Step 0: Persist everything to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')          # Skip this cell if you don't need persistence

# Choose a root dir that survives VM resets if you mounted Drive
ROOT = "/content/drive/MyDrive/ransomware_lab"  # or "/content" if Drive not mounted
!mkdir -p $ROOT

Mounted at /content/drive


##Step 1: Install Tools

In [2]:
%%bash
sudo apt-get update -qq
sudo apt-get install -y restic gnupg tree

Reading package lists...
Building dependency tree...
Reading state information...
gnupg is already the newest version (2.2.27-3ubuntu2.4).
gnupg set to manually installed.
Suggested packages:
  sphinx-rtd-theme-common
The following NEW packages will be installed:
  restic tree
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 7,439 kB of archives.
After this operation, 20.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 restic amd64 0.12.1-2ubuntu0.3 [7,391 kB]
Fetched 7,439 kB in 0s (18.7 MB/s)
Selecting previously unselected package tree.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Selecting previously unselected package restic.
Preparing to unpack .../restic_0.12.1-2ubuntu0.3_amd64.deb ...
Unpacking r

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [3]:
#Install extra utilities
%%bash
sudo apt-get install -y inotify-tools   # for near real‑time backup
pip install --quiet watchdog            # Python file‑watcher library

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libinotifytools0
The following NEW packages will be installed:
  inotify-tools libinotifytools0
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 54.8 kB of archives.
After this operation, 233 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libinotifytools0 amd64 3.22.1.0-2 [22.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 inotify-tools amd64 3.22.1.0-2 [32.2 kB]
Fetched 54.8 kB in 0s (328 kB/s)
Selecting previously unselected package libinotifytools0:amd64.
(Reading database ... 126424 files and directories currently installed.)
Preparing to unpack .../libinotifytools0_3.22.1.0-2_amd64.deb ...
Unpacking libinotifytools0:amd64 (3.22.1.0-2) ...
Selecting previously unselected package inotify-tools.
Preparing to unpack .../inotify-tools_3.22.1.0-2_amd64.d

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


##Step 2: Configure Restic Repo

In [4]:
import os, subprocess, textwrap, getpass, json, pathlib, shutil, time

# Change these if you like
DATA_DIR      = f"{ROOT}/victim_data"
RESTIC_REPO   = f"{ROOT}/backup_repo"
RESTORE_DIR   = f"{ROOT}/restore"
os.makedirs(DATA_DIR,   exist_ok=True)
os.makedirs(RESTORE_DIR, exist_ok=True)

# One‑liner to set the repo password (DON'T lose it!)
os.environ["RESTIC_PASSWORD"] = "colab‑demo‑super‑secret"  # choose a strong one IRL

# Initialize repository once
if not pathlib.Path(RESTIC_REPO, "config").exists():
    !restic -r $RESTIC_REPO init

created restic repository 99760c2fbb at /content/drive/MyDrive/ransomware_lab/backup_repo

Please note that knowledge of your password is required to access
the repository. Losing your password means that your data is
irrecoverably lost.


##Step 3: Generate sample "production" files

In [5]:
%%bash -s "$DATA_DIR"
TARGET=$1
mkdir -p "$TARGET/docs" "$TARGET/images"

# simple text data
echo "Quarterly revenue: \$123,456"  > "$TARGET/docs/report_Q1.txt"
echo "User list (PII redacted)"      > "$TARGET/docs/users.txt"

# simulate binary data
head -c 1M </dev/urandom > "$TARGET/images/raw_sensor_dump.bin"

tree -h "$TARGET"

[4.0K]  /content/drive/MyDrive/ransomware_lab/victim_data
├── [4.0K]  docs
│   ├── [  28]  report_Q1.txt
│   └── [  25]  users.txt
└── [4.0K]  images
    └── [1.0M]  raw_sensor_dump.bin

2 directories, 3 files


##Step 4: Define Helper to Print List of Snapshots

In [17]:
import subprocess
import sys
import os

def print_current_snapshots(restic_repo: str):
    """
    Prints the list of snapshots in the specified Restic repository.
    Relies on RESTIC_PASSWORD env var for authentication.
    """
    common_args = ["restic", "--repo", restic_repo]

    try:
        # Use --no-lock for faster listing if appropriate
        output = subprocess.check_output(
            common_args + ["snapshots", "--no-lock"],
            env=os.environ,
            text=True
        )
        print("🔍 Current snapshots in repository:")
        print(output)
    except subprocess.CalledProcessError as e:
        err = e.stderr if hasattr(e, 'stderr') and e.stderr else str(e)
        print("❗ Error listing snapshots:", err, file=sys.stderr)

##Step 5: Define Baseline Snapshot Helper

In [6]:
import subprocess
import json
import threading
import time
from datetime import datetime

In [7]:
def run(cmd: list[str]) -> subprocess.CompletedProcess:
    """
    Run the given command (printing it) and return a CompletedProcess
    (with .stdout/.stderr) for downstream parsing.
    """
    print(f"$ {' '.join(cmd)}")
    return subprocess.run(
        cmd,
        capture_output=True,
        text=True,
        check=True
    )

In [8]:
def get_baseline_snapshot_ids(restic_repo: str) -> list[str]:
    """
    Return a list of all snapshot IDs (short form) tagged 'baseline' in the repo.
    """
    proc = run(["restic", "-r", restic_repo, "snapshots", "--tag", "baseline", "--json"])
    snaps = json.loads(proc.stdout)
    # each snap has keys like "short_id", "time", etc.
    return [snap["short_id"] for snap in snaps]


In [9]:
def print_diff_details(diff: dict) -> None:
    """
    Pretty-print categorized diff results.
    """
    blocks = [
        ("➕ Added", diff.get("added", [])),
        ("✏️ Modified", diff.get("modified", [])),
        ("🗑️ Removed", diff.get("removed", [])),
        ("🧾 Metadata-only", diff.get("meta", [])),
    ]
    any_printed = False
    for title, items in blocks:
        if items:
            any_printed = True
            print(title + ":")
            for p in items:
                print(f"  • {p}")
    if not any_printed:
        print("🔍 No changes detected.")

In [12]:
import re
def get_diff_details(old_id: str, new_id: str, restic_repo: str) -> dict:
    """
    Parse `restic diff old new` and return categorized path lists.
    Supports both 'A/M/R/U path' and '+/- path' formats.
    """
    proc = run(["restic", "-r", restic_repo, "diff", old_id, new_id])

    added, modified, removed, meta = [], [], [], []

    for raw in proc.stdout.splitlines():
        line = raw.strip()
        if not line:
            continue
        # Skip headers/summaries
        if ":" in line and line.split(":", 1)[0].lower() in {
            "comparing", "files", "dirs", "added files", "removed files",
            "modified files", "added dirs", "removed dirs", "modified dirs"
        }:
            continue

        # Format 1: "A path", "M path", "R path", "U path"
        m = re.match(r"^([AMRU])\s+(.+)$", line)
        if m:
            code, path = m.group(1), m.group(2)
            if code == "A": added.append(path)
            elif code == "M": modified.append(path)
            elif code == "R": removed.append(path)
            elif code == "U": meta.append(path)
            continue

        # Format 2: "+ path" (added), "- path" (removed)
        m = re.match(r"^([+\-])\s+(.+)$", line)
        if m:
            code, path = m.group(1), m.group(2)
            if code == "+": added.append(path)
            elif code == "-": removed.append(path)
            continue

        # If we reach here it's an unrecognized line; ignore
        # print("DEBUG unparsed diff line:", line)

    return {"added": added, "modified": modified, "removed": removed, "meta": meta}


In [13]:
def ensure_baseline_snapshot(data_dir: str, restic_repo: str) -> None:
    """
    1) If no baseline exists, take one and return.
    2) Otherwise:
         a) Record old_id = latest baseline snapshot.
         b) Run `restic backup --tag baseline` to create new snapshot.
         c) Let new_id = ID of that new snapshot.
         d) Call get_diff_files(old_id, new_id):
            • if empty → forget/prune new_id (skip)
            • else    → print_changed_files(paths) and keep new_id
    """
    # 1) List existing baselines, pick latest by timestamp
    proc0 = run(["restic", "-r", restic_repo, "snapshots", "--tag", "baseline", "--json"])
    snaps = json.loads(proc0.stdout)
    snaps.sort(key=lambda s: s["time"])
    if not snaps:
        print("📦 No baseline found — creating initial baseline.")
        run(["restic", "-r", restic_repo, "backup", data_dir, "--tag", "baseline"])
        print(f"✅ Baseline created at {datetime.now().isoformat()}")
        return

    old_id = snaps[-1]["short_id"]

    # 2) Create the new snapshot
    print("⏳ Running backup to test for real changes…")
    proc = run(["restic", "-r", restic_repo, "backup", data_dir, "--tag", "baseline", "--json"])
    # parse the summary JSON for new_id
    new_id = None
    for line in proc.stdout.splitlines():
        try:
            msg = json.loads(line)
        except json.JSONDecodeError:
            continue
        if msg.get("message_type") == "summary":
            new_id = msg["snapshot_id"]
            break
    if new_id is None:
        raise RuntimeError("Could not find summary in restic JSON output")

    # 3) Diff old→new to get the **true** list of changed files
    # changed = get_diff_files(old_id, new_id, restic_repo)
    # if not changed:
    diff = get_diff_details(old_id, new_id, restic_repo)
    changed_count = sum(len(v) for v in diff.values())
    if changed_count == 0:
        print(f"⚠️ No real file-tree changes; forgetting baseline {new_id}")
        run(["restic", "-r", restic_repo, "forget", new_id, "--prune"])
        print("✅ Skipped baseline (no real changes).")
    else:
        print(f"✅ Real changes detected ({changed_count} paths); keeping {new_id}")
        print_diff_details(diff)


##Step 6: Define Periodic Baseline Scheduler

In [14]:
import threading
import time

def start_periodic_baseline(data_dir: str, restic_repo: str, interval: int):
    """
    Spawns a background thread that re-tags the latest state as a new baseline
    every `interval` seconds by calling ensure_baseline_snapshot().
    """
    stop_event = threading.Event()
    def loop():
        while True:
            ensure_baseline_snapshot(data_dir, restic_repo)
            # time.sleep(interval)
            stop_event.wait(interval)

    t = threading.Thread(target=loop, daemon=True)
    t.start()
    print()
    print(f"🕒 Started periodic baseline thread (every {interval}s)")
    return stop_event, t

# start_periodic_baseline(DATA_DIR, RESTIC_REPO, interval=15)
stop_ev, thread = start_periodic_baseline(DATA_DIR, RESTIC_REPO, interval=15)
time.sleep(5)

$ restic -r /content/drive/MyDrive/ransomware_lab/backup_repo snapshots --tag baseline --json
🕒 Started periodic baseline thread (every 15s)

📦 No baseline found — creating initial baseline.
$ restic -r /content/drive/MyDrive/ransomware_lab/backup_repo backup /content/drive/MyDrive/ransomware_lab/victim_data --tag baseline
✅ Baseline created at 2025-08-08T19:36:23.053861


##Step 7: Define Helper to Restore Latest Baseline Snapshot that is Readable

In [15]:
import subprocess
import os
import pathlib
import shutil
import sys
import json
import tempfile
from datetime import datetime

def restore_latest_clean_baseline(restic_repo: str, restore_dir: str):
    """
    Attempts to restore the latest clean baseline snapshot.
    Iterates through snapshots tagged 'baseline' in reverse chronological order,
    restores to a temporary directory, verifies critical files, moves data,
    and returns the restored snapshot ID.
    """
    print("🔍 Searching for clean baseline snapshots...")

    # 1. List baseline snapshots
    try:
        proc = subprocess.run(
            ["restic", "-r", restic_repo, "snapshots", "--tag", "baseline", "--json"],
            capture_output=True, text=True, check=True, env=os.environ
        )
        snapshots = json.loads(proc.stdout)
    except subprocess.CalledProcessError as e:
        print("❌ Failed to list baseline snapshots:", e.stderr.strip(), file=sys.stderr)
        return False, None

    if not snapshots:
        print("❌ No baseline snapshots found.")
        return False, None

    # 2. Sort by timestamp descending
    for snap in snapshots:
        snap["parsed_time"] = datetime.fromisoformat(snap["time"].rstrip("Z"))
    snapshots.sort(key=lambda s: s["parsed_time"], reverse=True)

    restore_path = pathlib.Path(restore_dir)
    tmp_root = pathlib.Path(tempfile.mkdtemp(prefix="restic_tmp_"))

    try:
        # 3. Try each snapshot until valid
        for snap in snapshots:
            sid = snap["id"]
            print(f"🕵️ Trying snapshot {sid}...")

            tmp_dir = tmp_root / sid
            if tmp_dir.exists():
                shutil.rmtree(tmp_dir)
            tmp_dir.mkdir(parents=True)

            # Restore
            try:
                subprocess.run(
                    ["restic", "-r", restic_repo, "restore", sid, "--target", str(tmp_dir)],
                    capture_output=True, text=True, check=True
                )
            except subprocess.CalledProcessError as e:
                print(f"⚠️ Restore failed for {sid}: {e.stderr.strip()}")
                continue

            # Locate victim_data
            victim_dirs = list(tmp_dir.rglob("victim_data"))
            if not victim_dirs:
                print(f"❌ victim_data not found in {sid}")
                continue
            victim_path = victim_dirs[0]

            # Verify critical file readability
            critical = victim_path / "docs" / "report_Q1.txt"
            if not critical.exists():
                print(f"❌ report_Q1.txt missing in {sid}")
                continue
            try:
                with open(critical, 'r') as f:
                    f.read(1)
            except Exception as e:
                print(f"❌ Cannot read report_Q1.txt in {sid}: {e}")
                continue

            # 4. Prepare restore_dir
            if restore_path.exists():
                shutil.rmtree(restore_path)
            restore_path.mkdir(parents=True)

            # Move contents
            for item in tmp_dir.iterdir():
                shutil.move(str(item), str(restore_path / item.name))

            print(f"✅ Restored clean baseline {sid} into {restore_dir}")
            return True, sid

        print("❌ No clean baseline snapshot found.")
        return False, None

    finally:
        shutil.rmtree(tmp_root)

##Step 8: 15 Seconds Pause

In [18]:
import time, subprocess, os
print("⌛ Sleeping 15s so real‑time activity accumulates …")
time.sleep(15)
print_current_snapshots(RESTIC_REPO)

⌛ Sleeping 15s so real‑time activity accumulates …
$ restic -r /content/drive/MyDrive/ransomware_lab/backup_repo snapshots --tag baseline --json
⏳ Running backup to test for real changes…
$ restic -r /content/drive/MyDrive/ransomware_lab/backup_repo backup /content/drive/MyDrive/ransomware_lab/victim_data --tag baseline --json
$ restic -r /content/drive/MyDrive/ransomware_lab/backup_repo diff d1579e53 41adb40a
⚠️ No real file-tree changes; forgetting baseline 41adb40a
$ restic -r /content/drive/MyDrive/ransomware_lab/backup_repo forget 41adb40a --prune
✅ Skipped baseline (no real changes).
🔍 Current snapshots in repository:
ID        Time                 Host          Tags        Paths
----------------------------------------------------------------------------------------------------------
d1579e53  2025-08-08 19:36:22  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
-------------------------------------------------------------------------------------------

##Step 8: Create a Unique Text File to Validate the Accuracy of Baseline Function

In [26]:
# 1. Show current baselines
print("▶️ Before Adding File:")
print_current_snapshots(RESTIC_REPO)

# 2. Create a dummy change
new_file = os.path.join(DATA_DIR, f"verify_{int(time.time())}.txt")
with open(new_file, "w") as f:
    f.write("trigger baseline\n")

print(f"📝 Created: {new_file}")

▶️ Before Adding File:
🔍 Current snapshots in repository:
ID        Time                 Host          Tags        Paths
----------------------------------------------------------------------------------------------------------
d1579e53  2025-08-08 19:36:22  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
b065179b  2025-08-08 19:37:29  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
51bad57a  2025-08-08 19:38:03  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
----------------------------------------------------------------------------------------------------------
3 snapshots

📝 Created: /content/drive/MyDrive/ransomware_lab/victim_data/verify_1754681909.txt


In [27]:
%%bash
VICTIM_PATH="/content/drive/MyDrive/ransomware_lab/victim_data"
tree -h "$VICTIM_PATH"

[4.0K]  /content/drive/MyDrive/ransomware_lab/victim_data
├── [4.0K]  docs
│   ├── [  28]  report_Q1.txt
│   └── [  25]  users.txt
├── [4.0K]  images
│   └── [1.0M]  raw_sensor_dump.bin
└── [  17]  verify_1754681909.txt

2 directories, 4 files


In [28]:
time.sleep(10)
# 3. Show baselines again
print("▶️ After Adding File:")
print_current_snapshots(RESTIC_REPO)

$ restic -r /content/drive/MyDrive/ransomware_lab/backup_repo snapshots --tag baseline --json
⏳ Running backup to test for real changes…
$ restic -r /content/drive/MyDrive/ransomware_lab/backup_repo backup /content/drive/MyDrive/ransomware_lab/victim_data --tag baseline --json
$ restic -r /content/drive/MyDrive/ransomware_lab/backup_repo diff 51bad57a 748f1b0d
✅ Real changes detected (1 paths); keeping 748f1b0d
➕ Added:
  • /content/drive/MyDrive/ransomware_lab/victim_data/verify_1754681909.txt
▶️ After Adding File:
🔍 Current snapshots in repository:
ID        Time                 Host          Tags        Paths
----------------------------------------------------------------------------------------------------------
d1579e53  2025-08-08 19:36:22  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
b065179b  2025-08-08 19:37:29  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
51bad57a  2025-08-08 19:38:03  d975cec45c61  baseline    /c

In [23]:
import os, glob, json, time
matches = glob.glob(os.path.join(DATA_DIR, "verify_*.txt"))
if not matches:
    print("❌ No verify_*.txt found. Create one first, then rerun.")
else:
  target = max(matches, key=os.path.getmtime)
  rel = os.path.relpath(target, DATA_DIR)
  print(f"🗂️ Target file to delete: {rel}")
  #Delete the file
  os.remove(target)
  print(f"🗑️ Deleted: {rel}")

🗂️ Target file to delete: verify_1754681848.txt
🗑️ Deleted: verify_1754681848.txt


In [24]:
%%bash
VICTIM_PATH="/content/drive/MyDrive/ransomware_lab/victim_data"
tree -h "$VICTIM_PATH"

[4.0K]  /content/drive/MyDrive/ransomware_lab/victim_data
├── [4.0K]  docs
│   ├── [  28]  report_Q1.txt
│   └── [  25]  users.txt
└── [4.0K]  images
    └── [1.0M]  raw_sensor_dump.bin

2 directories, 3 files


In [25]:
time.sleep(10)
print("▶️ After Deleting File:")
print_current_snapshots(RESTIC_REPO)

▶️ After Deleting File:
🔍 Current snapshots in repository:
ID        Time                 Host          Tags        Paths
----------------------------------------------------------------------------------------------------------
d1579e53  2025-08-08 19:36:22  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
b065179b  2025-08-08 19:37:29  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
51bad57a  2025-08-08 19:38:03  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
----------------------------------------------------------------------------------------------------------
3 snapshots



##Step 9: Simulate Ransomware attack

###Define Helper to Mark Simulated Ransomware Attack in Snapshots

In [29]:
import subprocess, os

def mark_attack_snapshot(data_dir: str, restic_repo: str):
    """
    Takes a Restic backup of data_dir and tags it 'attack' to mark
    the moment of the simulated ransomware event.
    """
    common_args = ["restic", "--repo", restic_repo]
    print("⚠️  Ransomware attack simulated — taking 'attack' snapshot…")
    subprocess.run(
        common_args + ["backup", data_dir, "--tag", "attack"],
        check=True, env=os.environ
    )
    print("✅  'attack' snapshot complete.")

###Encrypt the files and Simulate Attack

In [30]:
%%bash -s "$DATA_DIR"
VICTIM=$1
# Encrypt *.txt as a stand‑in for ransomware, then delete originals
cd "$VICTIM/docs"
for f in *.txt; do
  openssl enc -aes-256-cbc -md sha256 -salt -in "$f" -out "${f}.enc" -pass pass:evil123
  shred -u "$f"
done
echo "After attack:"
tree -h "$VICTIM"

After attack:
[4.0K]  /content/drive/MyDrive/ransomware_lab/victim_data
├── [4.0K]  docs
│   ├── [  48]  report_Q1.txt.enc
│   └── [  48]  users.txt.enc
├── [4.0K]  images
│   └── [1.0M]  raw_sensor_dump.bin
└── [  17]  verify_1754681909.txt

2 directories, 4 files


*** WARNING : deprecated key derivation used.
Using -iter or -pbkdf2 would be better.
*** WARNING : deprecated key derivation used.
Using -iter or -pbkdf2 would be better.


In [31]:
##mark the attack in Restic
mark_attack_snapshot(DATA_DIR, RESTIC_REPO)

⚠️  Ransomware attack simulated — taking 'attack' snapshot…
✅  'attack' snapshot complete.


In [33]:
time.sleep(10)
print_current_snapshots(RESTIC_REPO)

$ restic -r /content/drive/MyDrive/ransomware_lab/backup_repo snapshots --tag baseline --json
⏳ Running backup to test for real changes…
$ restic -r /content/drive/MyDrive/ransomware_lab/backup_repo backup /content/drive/MyDrive/ransomware_lab/victim_data --tag baseline --json
$ restic -r /content/drive/MyDrive/ransomware_lab/backup_repo diff 748f1b0d a6f24a69
✅ Real changes detected (4 paths); keeping a6f24a69
➕ Added:
  • /content/drive/MyDrive/ransomware_lab/victim_data/docs/report_Q1.txt.enc
  • /content/drive/MyDrive/ransomware_lab/victim_data/docs/users.txt.enc
🗑️ Removed:
  • /content/drive/MyDrive/ransomware_lab/victim_data/docs/report_Q1.txt
  • /content/drive/MyDrive/ransomware_lab/victim_data/docs/users.txt
🔍 Current snapshots in repository:
ID        Time                 Host          Tags        Paths
----------------------------------------------------------------------------------------------------------
d1579e53  2025-08-08 19:36:22  d975cec45c61  baseline    /content/d

##Step 10: Attempt to Read
Will fail

In [34]:
import pathlib, sys

plaintext = pathlib.Path(DATA_DIR) / "docs" / "report_Q1.txt"   # original file path

try:
    text = plaintext.read_text()          # should raise FileNotFoundError
    print("❌  UNEXPECTED: plaintext still readable!", text[:100])
    sys.exit(1)
except (FileNotFoundError, PermissionError):
    print("✅  Expected failure: plaintext is gone or unreadable — ransomware succeeded.")

✅  Expected failure: plaintext is gone or unreadable — ransomware succeeded.


##Step 11: Restore the Latest Clean Snapshot

In [35]:
#List all snapshots
print_current_snapshots(RESTIC_REPO)

🔍 Current snapshots in repository:
ID        Time                 Host          Tags        Paths
----------------------------------------------------------------------------------------------------------
d1579e53  2025-08-08 19:36:22  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
b065179b  2025-08-08 19:37:29  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
51bad57a  2025-08-08 19:38:03  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
748f1b0d  2025-08-08 19:38:36  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
0a4effda  2025-08-08 19:38:57  d975cec45c61  attack      /content/drive/MyDrive/ransomware_lab/victim_data
a6f24a69  2025-08-08 19:39:09  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
----------------------------------------------------------------------------------------------------------
6 snapshots



In [36]:
success, restored_id = restore_latest_clean_baseline(RESTIC_REPO, RESTORE_DIR)

🔍 Searching for clean baseline snapshots...
🕵️ Trying snapshot a6f24a6987ce986a3f77b842f624178d031d9ddf8bb2db058c3a06d5d203bf96...
❌ report_Q1.txt missing in a6f24a6987ce986a3f77b842f624178d031d9ddf8bb2db058c3a06d5d203bf96
🕵️ Trying snapshot 748f1b0deff596d06e79d61b09c2a6967d70d50a18e68235cce0c6e1fd96dfbe...
✅ Restored clean baseline 748f1b0deff596d06e79d61b09c2a6967d70d50a18e68235cce0c6e1fd96dfbe into /content/drive/MyDrive/ransomware_lab/restore


##Step 12: Verify Restored File Exists and Matches Expected Content

In [37]:
%%bash
# 1. Point to your restore directory
export RESTORE_DIR="/content/drive/MyDrive/ransomware_lab/restore"

# 2. Locate the nested victim_data folder
VICTIM_PATH=$(find "$RESTORE_DIR" -type d -name victim_data | head -n1)
if [ -z "$VICTIM_PATH" ]; then
  echo "❌ Could not find victim_data under $RESTORE_DIR"
  exit 1
fi

echo "✅ Restored data is at: $VICTIM_PATH"

# 3. List the docs you recovered
echo; echo "Recovered docs:"
tree -h "$VICTIM_PATH/docs" | sed -n '1,5p'

# 4. Show the contents of report_Q1.txt
echo; echo "Contents of report_Q1.txt:"
cat "$VICTIM_PATH/docs/report_Q1.txt"

✅ Restored data is at: /content/drive/MyDrive/ransomware_lab/restore/content/drive/MyDrive/ransomware_lab/victim_data

Recovered docs:
[4.0K]  /content/drive/MyDrive/ransomware_lab/restore/content/drive/MyDrive/ransomware_lab/victim_data/docs
├── [  28]  report_Q1.txt
└── [  25]  users.txt

0 directories, 2 files

Contents of report_Q1.txt:
Quarterly revenue: $123,456


In [38]:
import pathlib, hashlib, sys

# Adjust this to your actual restore path variable if needed
restore_root = pathlib.Path(RESTORE_DIR)
victim = next(restore_root.rglob("victim_data"), None)
if victim is None:
    print("❌  victim_data not found!"); sys.exit(1)

file = victim / "docs" / "report_Q1.txt"
if not file.exists():
    print("❌  report_Q1.txt missing!"); sys.exit(1)

content = file.read_text()
assert "Quarterly revenue: $123,456" in content, "Content mismatch!"

sha = hashlib.sha256(file.read_bytes()).hexdigest()
print(f"✅  report_Q1.txt verified — SHA‑256: {sha[:12]}…")

$ restic -r /content/drive/MyDrive/ransomware_lab/backup_repo diff a6f24a69 a616e857
✅  report_Q1.txt verified — SHA‑256: 2758452adf6e…


##Step 13: Compare Original to Restored
Sanity Check

In [39]:
%%bash -s "$DATA_DIR" "$RESTORE_DIR"
# $1 is the live data dir, $2 is where Restic put your restore tree
ORIG="$1"
RESTORE_ROOT="$2"

# 2. Locate the restored victim_data under RESTORE_ROOT
RESTORED_V=$(find "$RESTORE_ROOT" -type d -name victim_data | head -n1)
if [ -z "$RESTORED_V" ]; then
  echo "❌  Could not find victim_data under $RESTORE_ROOT"
  exit 1
fi

echo "✅  Restored data is at: $RESTORED_V"
echo

# 3. Compare original vs restored
echo "Comparing live vs restored:"
echo "  live:     $ORIG"
echo "  restored: $RESTORED_V"
echo

diff -rq "$ORIG" "$RESTORED_V" \
  && echo "✅  No unexpected differences." \
  || echo "🔶  Differences above are expected for encrypted docs."

✅  Restored data is at: /content/drive/MyDrive/ransomware_lab/restore/content/drive/MyDrive/ransomware_lab/victim_data

Comparing live vs restored:
  live:     /content/drive/MyDrive/ransomware_lab/victim_data
  restored: /content/drive/MyDrive/ransomware_lab/restore/content/drive/MyDrive/ransomware_lab/victim_data

Only in /content/drive/MyDrive/ransomware_lab/restore/content/drive/MyDrive/ransomware_lab/victim_data/docs: report_Q1.txt
Only in /content/drive/MyDrive/ransomware_lab/victim_data/docs: report_Q1.txt.enc
Only in /content/drive/MyDrive/ransomware_lab/restore/content/drive/MyDrive/ransomware_lab/victim_data/docs: users.txt
Only in /content/drive/MyDrive/ransomware_lab/victim_data/docs: users.txt.enc
🔶  Differences above are expected for encrypted docs.


##Optional: Prune Workspace
* Locating the deepest nested 'victim_data' directory inside restore_root
* Removing any '.enc' encrypted files
* Moving the restored data to the original victim_data_path
* Deleting all contents of the restore_root directory

In [40]:
import os
import shutil
import pathlib

def finalize_restoration(restore_root: str, victim_data_path: str):
    restore_root = pathlib.Path(restore_root)
    victim_data_path = pathlib.Path(victim_data_path)

    # Locate the deepest nested victim_data directory
    nested_victim_dirs = sorted(restore_root.rglob("victim_data"), key=lambda p: len(str(p)), reverse=True)
    if not nested_victim_dirs:
        print("❌ No victim_data directory found in restore root.")
        return False

    restored_victim = nested_victim_dirs[0]
    print(f"✅ Found restored victim_data at: {restored_victim}")

    # Remove encrypted files
    for enc_file in restored_victim.rglob("*.enc"):
        try:
            enc_file.unlink()
            print(f"🗑️ Deleted encrypted file: {enc_file}")
        except Exception as e:
            print(f"⚠️ Failed to delete {enc_file}: {e}")

    # Replace original victim_data with restored version
    if victim_data_path.exists():
        shutil.rmtree(victim_data_path)
    shutil.copytree(restored_victim, victim_data_path)
    print(f"📦 Moved restored data to: {victim_data_path}")

    # Delete all contents of the restore_root directory
    for item in restore_root.iterdir():
        try:
            if item.is_dir():
                shutil.rmtree(item)
            else:
                item.unlink()
            print(f"🧹 Deleted: {item}")
        except Exception as e:
            print(f"⚠️ Failed to delete {item}: {e}")

    return True

In [41]:
if success:
  finalize_restoration(
      restore_root=RESTORE_DIR,
      victim_data_path=DATA_DIR
  )

✅ Found restored victim_data at: /content/drive/MyDrive/ransomware_lab/restore/content/drive/MyDrive/ransomware_lab/victim_data
📦 Moved restored data to: /content/drive/MyDrive/ransomware_lab/victim_data
🧹 Deleted: /content/drive/MyDrive/ransomware_lab/restore/content


In [42]:
%%bash
#!/bin/bash

# Define the root directory
VICTIM_PATH="/content/drive/MyDrive/ransomware_lab/victim_data"
RESTORE_PATH="/content/drive/MyDrive/ransomware_lab/restore"

# Check if the directory exists
if [ ! -d "$VICTIM_PATH" ]; then
  echo "❌ Directory not found: $VICTIM_PATH"
  exit 1
fi
if [ ! -d "$RESTORE_PATH" ]; then
  echo "❌ Directory not found: $RESTORE_PATH"
  exit 1
fi

# Print the tree with human-readable sizes
echo "📁 Directory tree for: $VICTIM_PATH"
echo

tree -h "$VICTIM_PATH"
# Print the tree with human-readable sizes
echo "📁 Directory tree for: $RESTORE_PATH"
echo

tree -h "$RESTORE_PATH"

📁 Directory tree for: /content/drive/MyDrive/ransomware_lab/victim_data

[4.0K]  /content/drive/MyDrive/ransomware_lab/victim_data
├── [4.0K]  docs
│   ├── [  28]  report_Q1.txt
│   └── [  25]  users.txt
├── [4.0K]  images
│   └── [1.0M]  raw_sensor_dump.bin
└── [  17]  verify_1754681909.txt

2 directories, 4 files
📁 Directory tree for: /content/drive/MyDrive/ransomware_lab/restore

[4.0K]  /content/drive/MyDrive/ransomware_lab/restore

0 directories, 0 files


In [44]:
time.sleep(10)
print_current_snapshots(RESTIC_REPO)

🔍 Current snapshots in repository:
ID        Time                 Host          Tags        Paths
----------------------------------------------------------------------------------------------------------
d1579e53  2025-08-08 19:36:22  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
b065179b  2025-08-08 19:37:29  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
51bad57a  2025-08-08 19:38:03  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
748f1b0d  2025-08-08 19:38:36  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
0a4effda  2025-08-08 19:38:57  d975cec45c61  attack      /content/drive/MyDrive/ransomware_lab/victim_data
a6f24a69  2025-08-08 19:39:09  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
63c3895f  2025-08-08 19:39:43  d975cec45c61  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
----------------------------------------------

In [45]:
import threading
import time

stop_ev.set()
thread.join(timeout=5)
print("🛑 Periodic baseline stopped.")

🛑 Periodic baseline stopped.
